In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
from imblearn.under_sampling import RandomUnderSampler

Using TensorFlow backend.
/home/xuke/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xuke/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xuke/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xuke/anaconda3/lib/python3.7/site-packages/tensorflow/py

In [2]:
import imp
data_process=imp.load_source('data_process','../data_process.py')
from data_process import get_xml_data

## 读取数据

In [3]:
text,_,y=get_xml_data('../SMP2019/SMP2019_ECISA_Train.xml')

In [ ]:
# X = data[["text"]]
# y = data[["label"]]

## 分词

In [4]:
import jieba
def chinese_word_cut(mytext):
    return [" ".join(jieba.cut(temp)) for temp in mytext]

cuted_text = chinese_word_cut(text)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/home/xuke/anaconda3/lib/python3.7/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpsoxf05vl' -> '/tmp/jieba.cache'
Loading model cost 0.840 seconds.
Prefix dict has been built successfully.


In [5]:
cuted_text=np.array(cuted_text).reshape(len(cuted_text),1)

## 切分数据集

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cuted_text, y, random_state=1)

In [7]:
# X_train.shape
rus = RandomUnderSampler(random_state=0)
X_train, y_train= rus.fit_sample(X_train, y_train)

In [9]:
X_train

array([['老舍 笔下 的 “ 九曲十八弯 ” 属 中 俄 界河 额尔古纳河 水系 。'],
       ['到 售票口 买 完 门票 后 、 还 需要 坐 景区 大巴 前往 ！'],
       ['\\ \\ @ SarahMis   : 真的 是 很 卧槽 ！ ！'],
       ...,
       ['现在 没有 了 ， 已经 做过 三次 手术 ， 我 感觉 要犯 了'],
       ['老 觉得 无 烟味 ， 但 有 电池 味 … 应该 是 使用不当 …'],
       ['一个多月 前 ， 这个 自称 “ 不看 电影 ， 也 不 看电视 ， 只 看 英文 小说 ” 的 房地产 大亨 ， 出席 了 乐视 影业 姗姗来迟 的 年度 发布会 。']],
      dtype='<U314')

In [16]:
X_train[0]

array(['老舍 笔下 的 “ 九曲十八弯 ” 属 中 俄 界河 额尔古纳河 水系 。'], dtype='<U314')

In [10]:
def get_custom_stopword(stop_word_file):
    with open(stop_word_file,'rb') as f:
        stop_word = f.read()
        
    stop_word_list = stop_word.decode().split("/n")
    custom_stopword = [i for i in stop_word_list]
    return custom_stopword

stopwords = get_custom_stopword("哈工大停词表.txt")


# 建立模型

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
# term_matrix = DataFrame(vect.fit_transform(X_train.cuted_comment).toarray(), columns=vect.get_feature_names())
term_matrix = DataFrame(vect.fit_transform(X_train.ravel()).toarray(), columns=vect.get_feature_names())


In [14]:
term_matrix.shape

(8658, 23125)

In [16]:
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
# min_df = 0.00005 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。
min_df = 0.001

vect = CountVectorizer(max_df = max_df,
                       min_df = min_df,
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                       stop_words=frozenset(stopwords))

term_matrix = DataFrame(vect.fit_transform(X_train.ravel()).toarray(), columns=vect.get_feature_names())


In [17]:
term_matrix.shape

(8658, 1166)

In [18]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()


In [19]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(vect, nb)
pipe


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_df=0.8, min_df=0.001,
                                 stop_words=frozenset({'———\r\n'
                                                       '》），\r\n'
                                                       '）÷（１－\r\n'
                                                       '”，\r\n'
                                                       '）、\r\n'
                                                       '＝（\r\n'
                                                       ':\r\n'
                                                       '→\r\n'
                                                       '℃ \r\n'
                                                       '&\r\n'
                                                       '*\r\n'
                                                       '一一\r\n'
                                                       '~~~~\r\n'
                                                       '’\r\n'
       

In [21]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

/home/xuke/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', 'sub', 'sup', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｃ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '一一', '一个', '一些', '一切', '一则', '一方面', '一旦', '一来', '一样', '一番', '一直', '一般', '万一', '上下', '不仅', '不但', '不光', '不单', '不只', '不如', '不怕', '不惟', '不成', '不拘', '不比', '不然', '不特', '不独', '不管', '不论', '不过', '不问', '与其', '与否', '与此同时', '两者', '为了', '为什么', '为何', '为着', '乃至', '之一', '之所以', '之类', '乌乎', '也好', '也就是说', '也罢', '于是', '于是乎', '云云', '人家', '什么', '什么样', '从而', '他人', '他们', '以便', '以免', '以及', '以至', '以至于', '以致', '任何', '任凭', '似的', '但是', '何况',

nan

In [24]:
# pipe.fit(X_train.cuted_comment, y_train)
pipe.fit(X_train.ravel(), y_train)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_df=0.8, min_df=0.001,
                                 stop_words=frozenset({'———\r\n'
                                                       '》），\r\n'
                                                       '）÷（１－\r\n'
                                                       '”，\r\n'
                                                       '）、\r\n'
                                                       '＝（\r\n'
                                                       ':\r\n'
                                                       '→\r\n'
                                                       '℃ \r\n'
                                                       '&\r\n'
                                                       '*\r\n'
                                                       '一一\r\n'
                                                       '~~~~\r\n'
                                                       '’\r\n'
       

In [26]:
y_pred = pipe.predict(X_test.ravel())

# 评价指标

In [27]:
from sklearn import metrics
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score


In [34]:
ACU=accuracy_score(y_test, y_pred)
# p=precision_score(y_test, y_pred)
# r=recall_score(y_test, y_pred)
# f1=f1_score(y_test, y_pred)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [31]:
# print("The accruacy score is %.3f   f1_score: %.3f precision_score: %.3f recall_score: %.3f " % (ACU,f1,p,r))

NameError: name 'f1' is not defined

In [35]:
ACU=accuracy_score(y_test, y_pred)
# p=precision_score(y_test, y_pred,pos_label=0)
# r=recall_score(y_test, y_pred,pos_label=0)
# f1=f1_score(y_test, y_pred,pos_label=0)

In [36]:
ACU

0.654574986464537

In [ ]:
print("The accruacy score is %.3f   f1_score: %.3f precision_score: %.3f recall_score: %.3f " % (ACU,f1,p,r))

In [32]:
metrics.confusion_matrix(y_test, y_pred)

array([[1265,  286,  165],
       [ 220,  520,  202],
       [ 168,  235,  633]])